In [1]:
import drjit as dr
import mitsuba as mi

import os
import sys
ROOT_DIR = os.path.dirname(os.getcwd())
sys.path.append(ROOT_DIR)

from utils.utils import image_to_bm, to_float, unidim_to_bm, rel_l1_loss, mse, LOSS_FNS

mi.set_variant('cuda_ad_rgb')

In [2]:
KEY = 'green.reflectance.value'

In [9]:
def get_param_variance(theta_val=0.1, spp=128, spp_moments=1):
    scene = mi.load_file('../scenes/cbox.xml')
    params = mi.traverse(scene)
    image_ref = mi.render(scene, params, spp=spp)
    
    theta = mi.Float(theta_val)
    dr.enable_grad(theta)
    params[KEY] = mi.Color3f(
        0.2 * theta,
        0.5 * theta,
        0.8 * theta
    )
    params.update()

    # estimate gradients
    image = mi.render(scene, params, spp=spp)
    dr.backward(mse(image, image_ref))
    grad = dr.grad(theta)

    # estimate moments
    moments2 = None
    for _ in range(spp // spp_moments):
        image_var = mi.render(scene, params, spp=spp_moments)
        dr.backward(mse(image_var, image_ref))
        grad2 = dr.grad(theta)
        if moments2 is None:
            moments2 = dr.sqr(grad2)
        else:
            moments2 += dr.sqr(grad2)
    moments2 /= spp // spp_moments

    # compute variance
    variance = moments2 - dr.sqr(grad)
    return to_float(variance), to_float(grad), to_float(moments2), image_to_bm(image), image_to_bm(image_ref)

In [10]:
variance, grad, moments2, img, img_ref = get_param_variance(0.8)
variance, grad, moments2

(1.8189894035458565e-12, 0.0013304746244102716, 1.7701645447232295e-06)

In [11]:
img

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

In [12]:
img_ref

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

In [13]:
variance, grad, moments2, img, img_ref = get_param_variance(0.0)
variance, grad, moments2

(1.4779288903810084e-12, -0.0013571204617619514, 1.8417774754198035e-06)

In [14]:
img

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]